# Homework 3 

You will have 2 datasets to work on. 
#### You have to answer questions in this [form](https://goo.gl/forms/5gfxvKZxpoydoeOB2) and provide your code

### 1. Wikipedia Web Traffic Time Series

Data from [Kaggle competition](https://www.kaggle.com/c/web-traffic-time-series-forecasting)* )

*wikipedia_train3* - train data *wikipedia_test3* - test data created by us from original train data . For more information about dataset, please visit Homework1 assignment


### 2. French dataset

Target variable is prime_tot_ttc – house of insurance price.   
The variables in the files are:   
code_postal: postal code of the insured property   
nb_pieces: number of rooms   
veranda: presence of a veranda   
biens_nomades: ceiling of coverage of goods (laptop, tablets, musical instruments ,…)   
sit_fam: family situation of the insured   
profession: profession of the insured   
nature_lri: nature of the insured property    
societe: company covering the risk   
cdomact: area of activity of the insured   
niv_urbanisation: level of urbanization in which is the insured property   
age_societaire: age of the member   
objets_precieux: presence of precious objects in the insured property   
pres_contrat_auto: holding a contract of auto insurance   
formule: guarantee formula   
sit_juridique: legal situation   
surf_habitat: area of insured property   
patrimoine_mob: amount of heritage insured furniture   
var1 - ... - var11: non-explicit variables   


## Wikipedia page views (SMAPE metric)

In [353]:
import pandas as pd
import numpy as np
from sklearn.model_selection import KFold, GridSearchCV

In [336]:
### This is data from wikipedia3 archive in data folder
train = pd.read_csv("../data/wikipedia_train3.csv", parse_dates=['date'])
test = pd.read_csv("../data/wikipedia_test3.csv", parse_dates=['date'])

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. **For _only_ train** create a holdout validation using any type of split you think is useful here. What is the split type you are using? Answer in google forms


In [356]:
train_train = train[train['date'] < '2016-07-01']
train_test = train[train['date'] >= '2016-07-01']
train['date'].min()

Timestamp('2016-01-01 00:00:00')

In [236]:
sites = train_train['Page'].drop_duplicates()
means = {}

for i in sites:
    page = train_train[train_train['Page'] == i]
    means[i] = page[page['date'] > page['date'].max() - relativedelta(days=15)].median()

In [285]:
data_medians = pd.DataFrame([[i, means[i][0]] for i in means.keys()])

**2.** Write a code to compare the score of your validation and test set. For scoring, use metric SMAPE (code is in lecture). For prediction use 15 previous days median. In the google form write your validation score.

In [331]:
import math

def smape_fast(y_true, y_pred):
    out = 0
    for i in range(y_true.shape[0]):
        a = y_true.iloc[i]
        b = y_pred[i]
        if b < 1:
            b = 0
        c = a + b
        if c == 0:
            continue
        out += math.fabs(a - b) / c
    out *= (200.0 / y_true.shape[0])
    return out

In [344]:
smape_fast(train_test['Visits'], list(data_medians[1]) * int(train_test.shape[0] / len(data_medians[1])))

41.016810453819595

**3.** Perform K-fold validation using your type of split. Run GridSearch with any classificator you like and set of parameters to optimize, providing it with your custom validation.  Compare the score of your validation and test set. For scoring, again, use metrics SMAPE. In the google form write your scores on validation and test sets

In [352]:
def create_validation(df, start_date):
    return df.loc[(df['date'] >= pd.to_datetime(start_date) - relativedelta(days=0)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=2))].index, \
           df.loc[(df['date'] >= pd.to_datetime(start_date) + relativedelta(months=2)) & \
                  (df['date'] <  pd.to_datetime(start_date) + relativedelta(months=4))].index
    
kf =  KFold(n_splits=4, shuffle=False, random_state=42)
gscv = GridSearchCV()

In [377]:
myCViterator = []
for i in range(1, 4, 2):
    trainIndices, valIndices = create_validation(train_train, train_train['date'].min() + relativedelta(months=i))
    myCViterator.append( (trainIndices, valIndices) )

In [378]:
### Your validation iterator
for x,y in myCViterator:
    print (min(x), min(y))

0 366265
0 1075165


## French house insurance prediction (MAPE metric)

**1.** Take a look carefuly at train and test dataset. Note for you what is the difference in them and how they are dependent. You may use any tool you like, any EDA you want. Decribe your validation and reason for your choice in google form.

**2.** 
+ Implement your validation. 
+ Generate additional features. 
+ Build a model you like/you can. 
+ Wait for training your model. If it takes too much time, listen to https://www.youtube.com/watch?v=XWkGL4eZR3E
+ Submit your scores on validation and test to google form. Use MAPE metric for scoring. 
+ Ланфрен ланфра лантатита

In [734]:
import pandas_profiling as pp

train = pd.read_csv('../../source/data/train_french.csv')
test = pd.read_csv('../../source/data/test_french.csv')

train.drop(['profession', 'code_postal'], axis=1, inplace=True)
test.drop(['profession', 'code_postal'], axis=1, inplace=True)

In [735]:
nature_lri_dict = {}
nature_lri_arr = train.nature_lri.unique()

for i in range(len(nature_lri_arr)):
    nature_lri_dict[nature_lri_arr[i]] = i
    
niv_urbanisation_dict = {}
niv_urbanisation_arr = train.niv_urbanisation.unique()
for i in range(len(niv_urbanisation_arr)):
    niv_urbanisation_dict[niv_urbanisation_arr[i]] = i

In [736]:
train.veranda = train.veranda.map({"NON" : 0, "OUI": 1})
test.veranda = test.veranda.map({"NON" : 0, "OUI": 1})

train.sit_fam = train.sit_fam.map({"Indivividu" : 0, "Famille": 1})
test.sit_fam = test.sit_fam.map({"Indivividu" : 0, "Famille": 1})

train.var9 = train.var9.map({"NON" : 0, "OUI": 1})
test.var9 = test.var9.map({"NON" : 0, "OUI": 1})

train.nature_lri = train.nature_lri.map(nature_lri_dict)
test.nature_lri = test.nature_lri.map(nature_lri_dict)


# train.profession = train.profession.map(profession_dict)
# test.profession = test.profession.map(profession_dict)


train.niv_urbanisation = train.niv_urbanisation.map(niv_urbanisation_dict)
test.niv_urbanisation = test.niv_urbanisation.map(niv_urbanisation_dict)

In [737]:
# train.var9.value_counts()[0] + train.var9.value_counts()[1]
# len(train.var9)

# train.var7.unique()
# train.var7.value_counts()


# test.profession.unique()
# train.profession.unique()

# train2 = train + pd.get_dummies(train['nb_pieces'], prefix='nb_pieces')
# np.array(pd.get_dummies(test['nb_pieces'], prefix='nb_pieces'))
# train2.drop(['nb_pieces'], axis=0, inplace=True)

# print(len(y_test))
# print(len(y))
# print(train2.info())

In [738]:
# print(train.info())

In [785]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(train.drop('prime_tot_ttc', axis=1), 
                                                   train['prime_tot_ttc'],
                                                   test_size=0.33, 
                                                   random_state=42)

y_train = y_train.astype(int)
y_test = y_test.astype(int)

x_test_origin = test.drop(['prime_tot_ttc'], axis = 1)
y_test_origin = test.prime_tot_ttc.astype(int)

In [786]:
x_test.shape

(66000, 26)

In [787]:
max_depth_values = range(14, 16)
max_features_values = range(6, 7)
forest_params = {'max_depth': max_depth_values,
                 'max_features': max_features_values}

In [788]:
x_train.shape
y_train.shape

(134000,)

In [789]:
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier, BaggingRegressor
forest = RandomForestClassifier(random_state=17)
gsv = GridSearchCV(forest, forest_params, n_jobs = -1, verbose=1)
gsv.fit(x_train, y_train)
print (gsv.best_params_)
# print (gsv.best_score_)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


/Users/alatyshe/virt/lib/python3.6/site-packages/sklearn/model_selection/_split.py:605: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   14.3s remaining:    0.0s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   14.3s finished


{'max_depth': 14, 'max_features': 6}


In [791]:
print (gsv.best_score_)

0.08870149253731344


In [792]:
y_predict_test = gsv.predict(x_test)

In [793]:
y_predict_origin = gsv.predict(x_test_origin)
# X_test

In [760]:
from sklearn.utils import check_array
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [761]:
mean_absolute_percentage_error(y_test, y_predict_test)

19.79811155013306

In [ ]:

mean_absolute_percentage_error(y_test_origin, hah)

In [629]:
pp.ProfileReport(train)

Number of variables,29
Number of observations,200000
Total Missing (%),0.0%
Total size in memory,44.3 MiB
Average record size in memory,232.0 B
Numeric,17
Categorical,2
Boolean,10
Date,0
Text (Unique),0
Rejected,0


In [630]:
# pp.ProfileReport(test)

In [ ]:
# s = train.set_index(train.Page.ffill()).Visits
# c = s.groupby(level=0).cumcount() + 1
# s.index = [s.index, c]
# s = s.unstack()
# s.columns =  np.unique(np.array(train['date']))
# s['haha'] = [i for i in range(s.shape[0])]